In [3]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import scipy.signal as signalpy
import wave
import sys
import math
import contextlib
import IPython
import os
import librosa
import librosa.display
import pandas as pd
import sklearn.datasets as datasets
import shutil
import scipy
from scipy.interpolate import interp1d
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


### Loading pandas dataframe from .npy file saved previously

In [12]:
pcg_dataframe = pd.DataFrame(data = np.load('arrays/pcg_dataframe_binarized_padded.npy' , allow_pickle=True) , columns = [
                                'Patient ID', 'Locations', 'Murmur', 'Murmur locations',
                                 'Most audible location', 'Systolic murmur timing',
                                 'Systolic murmur shape', 'Systolic murmur grading',
                                 'Systolic murmur pitch', 'Systolic murmur quality',
                                 'Diastolic murmur timing', 'Diastolic murmur shape',
                                 'Diastolic murmur grading', 'Diastolic murmur pitch',
                                 'Diastolic murmur quality', 'Additional ID',
                                 'current_auscultation_location	', 'signal'])
display(pcg_dataframe)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,2530,AV+PV+TV+MV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TV,"[0.044283557683229446, 0.08357305079698563, 0...."
1,2530,AV+PV+TV+MV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PV,"[0.03783801943063736, 0.06029642000794411, 0.0..."
2,2530,AV+PV+TV+MV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MV,"[0.18855690956115723, 0.042485110461711884, -0..."
3,2530,AV+PV+TV+MV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AV,"[0.006233137100934982, -0.005920783616602421, ..."
4,9979,AV+PV+TV+MV,1,AV+MV+PV+TV,TV,Holosystolic,Diamond,III/VI,High,Harsh,NaN,NaN,NaN,NaN,NaN,NaN,TV,"[-0.1001439318060875, -0.8237980604171753, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,85345,AV+PV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PV,"[0.00654612248763442, 0.017596282064914703, 0...."
3003,85345,AV+PV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AV,"[0.03042028099298477, 0.00832417979836464, -0...."
3004,85349,AV+PV+TV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AV,"[0.0020578710827976465, -0.008989883586764336,..."
3005,85349,AV+PV+TV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PV,"[0.0742131769657135, 0.05925178900361061, 0.00..."
